<a href="https://colab.research.google.com/github/SashaNasonova/burnSeverity/blob/2025-updates/BurnSeverityMapping-DataSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Burn Severity Mapping Notebook
This notebook is intended to be used for small scale, interactive burn severity mapping of individual fires. For large scale semi-automated mapping please refer to the main python scripts (https://github.com/SashaNasonova/burnSeverity).

The methodology is based on the Burned Area Reflectance Classification (BARC) product developed by the USGS that aims to estimate burn severity through a spectral comparison of pre- and post-fire medium resolution (20 - 30m) satellite imagery.

Healthy vegetation reflects strongly in the near-infrared (NIR) portion of the electromagnetic spectrum whereas rock and bare soil reflects strongly in the mid to shortwave infrared (SWIR) portion. In other words, healthy vegetation reflects strongly in NIR and reflects weakly in SWIR **(↑NIR,↓SWIR)**, whereas soil, bare rock and burned woody vegetation reflect strongly in SWIR and weakly in NIR **(↑SWIR,↓NIR)**. This inverse relationship can be leveraged to provide an estimate of burn severity where both pre- and post-fire imagery is available.

The Normalized Burn Ratio (NBR) is a spectral index that captures the relationship between NIR and SWIR bands. The difference between pre- and post-fire NBR (dNBR) can then be used to quantify wildfire burn severity (**↑dNBR ∝ ↑Severity**) using the following equations.

(1) NBR = (NIR - SWIR) / (NIR + SWIR) \\
(2) dNBR = NBRpre - NBRpost

Once dNBR has been calculated, it can be transformed into a burn severity classification product using a variety of methods ranging from simple thresholding to more complex supervised classifications informed by ground observations. This process is based on the USGS BARC256 methodology which scales the data to an 8-bit representation and utilizes static thresholds (76,110,187) to create a burn severity classification from the dNBR raster.

This notebook was designed to assess image availability prior to mapping using BurnSeverityMapping.ipynb

Before beginning, please ensure that you are registered to use Google Earth Engine and have the Google Earth Engine API enabled as part of a Google Cloud project. If that is not the case, please follow the instructions on getting started (https://github.com/SashaNasonova/burnSeverity/blob/main/Getting_Started_with_GEE.md). You will need to take note of the project id and enter it later on.



---



### **Part 1: Set-up**

In [1]:
# Clone github repository to be able to access the test data and provincial extent vector data
!git clone https://github.com/SashaNasonova/burnSeverity.git

Cloning into 'burnSeverity'...
remote: Enumerating objects: 536, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 536 (delta 82), reused 38 (delta 17), pack-reused 386 (from 1)
Receiving objects: 100% (536/536), 270.13 MiB | 13.13 MiB/s, done.
Resolving deltas: 100% (255/255), done.
Updating files: 100% (236/236), done.


In [2]:
# Install the libraries
%pip install geemap #removed version specification
%pip install pycrs rasterio python-pptx cartopy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 10.4 MB/s eta 0:00:00
  Created wheel for pycrs: filename=PyCRS-1.0.2-py3-none-any.whl size=32687 sha256=d9dfc20110f2622978b45f02d92fb74b8a3bf0c4d0230755bffe4350f650d098
  Stored in directory: /root/.cache/pip/wheels/5f/ad/a3/183ed754d7698fc15a2eb153705e05d05a0d97f3331293ce48
Successfully built pycrs


In [3]:
# Import the libraries
import ee
import geemap
print('geemap version',geemap.__version__)
import os, json, shutil
import geopandas
from osgeo import gdal
from google.colab import files

import rasterio
from rasterio.plot import show
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import numpy as np
import pandas as pd
from pathlib import Path

import cartopy.crs as ccrs
import cartopy.feature as cfeature

from pptx import Presentation
from pptx.util import Cm, Inches
from pptx.util import Pt

geemap version 0.35.2


### **Part 2: Google Earth Engine authentication and initialization**

Authenticate and intialize GEE. After running the cell below, a sign-in window will pop-up. Please follow prompts to authenticate (sign-in, continue and continue).

In [4]:
#Authenticate gee
ee.Authenticate()

 Please note, the **Project ID** may be something other than the project name (ex. burn-severity-2024) and may contain additional numbers (ex. burn-severity-2024-456181). Make sure to copy the actual **Project ID** and enter it in the cell below.

In [5]:
# Initialize with a google cloud project
project = 'burn-severity-2024'
ee.Initialize(project=project)



---



### **Part 3: Fire perimeter import and visualization**

The code below will load in a polygon shapefile and display the attribute table. The fire_shp variable is the path to your dataset.

In [6]:
# Open fires shapefile
fires_shp = 'burnSeverity/test/vectors/K70910.shp'
fires = geemap.shp_to_ee(fires_shp)

# Visualize in table format
fires_df = geopandas.read_file(fires_shp)
fires_df

/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Field TRACK_DATE create as date field, though DateTime requested.
  ogr_write(
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Field LOAD_DATE create as date field, though DateTime requested.
  ogr_write(
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Value 279718852.108900011 of field FEATURE_AR of feature 0 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(


,FIRE_NUMBE,VERSION_NU,FIRE_YEAR,FIRE_SIZE_,SOURCE,TRACK_DATE,LOAD_DATE,FIRE_STATU,FIRE_URL,FEATURE_CO,FEATURE_AR,FEATURE_LE,OBJECTID,SHAPE_AREA,SHAPE_LEN,geometry
0,K70910,2024082001,2024,27971.9,Processed IR image,2024-08-20,2024-08-20,Being Held,https://wildfiresituation.nrs.gov.bc.ca/incide...,JA70003000,2.797189e+08,137780.3225,9165357,0.0,0.0,"POLYGON ((1320010.372 620243.186, 1320041.101 ..."


Now visualize spatially.

In [7]:
Map = geemap.Map()
Map.addLayer(fires,{},'Fire Polys')
Map.centerObject(fires,zoom=8)
Map

Map(center=[50.5247667490912, -121.40723329369156], controls=(WidgetControl(options=['position', 'transparent_…



---



### **Part 4: Individual fire perimeter selection (1 fire perimeter)**

Select one fire perimeter by fire number. Please make sure that the **fieldname** variable matches your dataset.

In [8]:
# Now select one fire (in the test data, there's only one fire perimeter)
firenumber = 'K70910' #change fire name
fieldname = 'FIRE_NUMBE' #unique firenumber field, change if needed

# First check if the firenumber exists in the shapefile provided
firelist = fires_df[fieldname].tolist()

if firenumber not in firelist:
  print('Selected fire number:',firenumber)
  print('Available fire numbers: ',firelist)
  raise ValueError('Fire number not in fire list. Typo?')

# Create output folder
if not os.path.exists(firenumber):
  os.mkdir(firenumber)

# Save a copy of the fire perimeter
vector_folder = os.path.join(firenumber,'vectors')
if not os.path.exists(vector_folder):
  os.mkdir(vector_folder)

outshp = os.path.join(vector_folder,firenumber+'.shp')
fires_df_sub = fires_df[fires_df[fieldname]==firenumber]
fires_df_sub.to_file(outshp,driver='ESRI Shapefile')

# Load in the single perimeter
poly = geemap.shp_to_ee(outshp)




---



In [21]:
def run_search(col,dattype,col_size):
  def getDate(im):
        return(ee.Image(im).date().format("YYYY-MM-dd"))

  def getSceneIds(im):
      return(ee.Image(im).get('PRODUCT_ID'))

  def mosaicByDate(indate):
      d = ee.Date(indate)
      #print(d)
      im = col.filterBounds(poly).filterDate(d, d.advance(1, "day")).mosaic()
      #print(im)
      return(im.set("system:time_start", d.millis(), "system:index", d.format("YYYY-MM-dd")))

  def runDateMosaic(col_list):
      #get a list of unique dates within the list
      date_list = col_list.map(getDate).getInfo()
      udates = list(set(date_list))
      udates.sort()
      udates_ee = ee.List(udates)

      #mosaic images by unique date
      mosaic_imlist = udates_ee.map(mosaicByDate)
      return(ee.ImageCollection(mosaic_imlist))

  def get_cloud(img1):
    ### Change as of Oct 24, 2023: cloud shadow is too inaccurate, remove
    ### Though it is picking up topographic shadow. Questions!
    # Bits 3 and 4 are cloud and cloud shadow, respectively.
    #cloudShadowBitMask = (1 << 4)
    cloudBitMask = (1 << 3)
    # Get the pixel QA band.
    qa = img1.select('QA_PIXEL')
    #set both flags to 1
    #clouds = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cloudShadowBitMask).eq(0)).rename('cloudmsk')
    clouds = qa.bitwiseAnd(cloudBitMask).eq(0).rename('cloudmsk')
    return(img1.addBands(clouds))


  col_list = col.toList(col_size)

  # Create mosaics
  col_mosaic = runDateMosaic(col_list)

  # Ask server for individual scene metadata
  metadata = col.getInfo()

  # Turn metadata into table format
  features = metadata['features']

  out = []
  for i in features:
      d1 = pd.DataFrame([{'id':i['id']}])
      p1 = pd.DataFrame([i['properties']])
      t1 = d1.join(p1)
      out.append(t1)

  meta_df = pd.concat(out)

  def strDate(string):
      u_str = string.rsplit('_')[1].rsplit('T')[0]
      s = u_str[0:4] + '-' + u_str[4:6] + '-' + u_str[6:8]
      return(s)

  #add date column
  if dattype.startswith('S2'):
      meta_df['date'] = meta_df['DATATAKE_IDENTIFIER'].apply(strDate)
  else:
      meta_df['date'] = meta_df['DATE_ACQUIRED']

  #outpath = os.path.join(outfolder,'pre_sceneMetadata.csv')
  #meta_df.to_csv(outpath)

  #make a copy of meta_df
  meta_scenes = meta_df.copy()

  # Classify to get coverage and cloud extent, fix this to check if any bands are equal to 0
  def classify_extent(img1):
      if dattype.startswith('S2'):
          classes = img1.expression("((B2 + B3 + B4) !=0) ? 1 "
                                      ": 0",{'B2': img1.select('B2'),
                                            'B3': img1.select('B3'),
                                            'B4': img1.select('B4')}).rename('c').clip(poly)
      else:
          classes = img1.expression("((B2 + B3 + B4) !=0) ? 1 "
                                      ": 0",{'B2': img1.select('SR_B2'),
                                            'B3': img1.select('SR_B3'),
                                            'B4': img1.select('SR_B4')}).rename('c').clip(poly)
      return(classes)

  mosaic_extent = col_mosaic.map(classify_extent).toBands()

  ## Classify cloud cover
  def classify_cc(img1):
      if dattype.startswith('S2'):
          classes = img1.expression("(MSK_CLDPRB > 30) ? 1 "
                              ": 0",{'MSK_CLDPRB': img1.select('MSK_CLDPRB')}).rename('c').clip(poly)
      else:
          classes = img1.expression("(cloudmsk == 1) ? 0 "
                              ": 1",{'cloudmsk': img1.select('cloudmsk')}).rename('c').clip(poly)
      return(classes)

  if dattype.startswith('S2'):
      mosaic_cc = col_mosaic.map(classify_cc).toBands()
      aot = col_mosaic.select('AOT').toBands().divide(1000)
      reduced_mean_aot = aot.reduceRegion(reducer=ee.Reducer.mean(),geometry=poly.geometry(),maxPixels=100000000000,scale=30).getInfo()
  else:
      mosaic_cloudmsk = col_mosaic.map(get_cloud)
      mosaic_cc = mosaic_cloudmsk.map(classify_cc).toBands()

  #Calculate statistics, if the image is too big this may fail.
  #This step causes problems sometimes due to maxPixels limits
  reduced_sum = mosaic_extent.reduceRegion(reducer=ee.Reducer.sum(),geometry=poly.geometry(),scale=30,maxPixels=100000000000).getInfo()
  reduced_count = mosaic_extent.reduceRegion(reducer=ee.Reducer.count(),geometry=poly.geometry(),maxPixels=100000000000,scale=30).getInfo()

  reduced_sum_cc = mosaic_cc.reduceRegion(reducer=ee.Reducer.sum(),geometry=poly.geometry(),maxPixels=100000000000,scale=30).getInfo()
  reduced_count_cc = mosaic_cc.reduceRegion(reducer=ee.Reducer.count(),geometry=poly.geometry(),maxPixels=100000000000,scale=30).getInfo()

  #Rearrange and calculate percent coverage and percent cloud cover
  #extent
  df_sum = pd.DataFrame([reduced_sum]).T
  df_sum.columns = ['sum']

  df_count = pd.DataFrame([reduced_count]).T
  df_count.columns = ['count']

  df_perc = df_sum.join(df_count)
  df_perc['percent_coverage'] = (df_perc['sum']/df_perc['count'])*100

  #cloud cover
  df_sum_cc = pd.DataFrame([reduced_sum_cc]).T
  df_sum_cc.columns = ['sum_cc']

  df_count_cc = pd.DataFrame([reduced_count_cc]).T
  df_count_cc.columns = ['count_cc']

  df_perc_cc = df_sum_cc.join(df_count_cc)
  df_perc_cc['percent_cc'] = (df_perc_cc['sum_cc']/df_perc_cc['count_cc'])*100
  #print(df_perc_cc)

  if dattype.startswith('S'):
      #aot
      df_mean_aot = pd.DataFrame([reduced_mean_aot]).T
      df_mean_aot.columns = ['mean_aot']

      #join extent and cc
      meta_df_ext_temp = df_perc.join(df_perc_cc)

      #get rid of cc suffix
      oldnames = meta_df_ext_temp.index
      newnames = [s.rsplit('_')[0] for s in oldnames]
      meta_df_ext_temp.index = newnames

      #get rid of aot suffix
      oldnames = df_mean_aot.index
      newnames = [s.rsplit('_')[0] for s in oldnames]
      df_mean_aot.index = newnames

      meta_df_ext = meta_df_ext_temp.join(df_mean_aot)
      #print(meta_df_ext)

  else:
      #join extent and cc
      meta_df_ext = df_perc.join(df_perc_cc)

      #get rid of cc suffix
      oldnames = meta_df_ext.index
      newnames = [s.rsplit('_')[0] for s in oldnames]
      meta_df_ext.index = newnames

  #get average scene cloud cover and join to mosaic metadata
  if dattype.startswith('S2'):
      meta_scenes_cld = meta_scenes.groupby('date')['CLOUDY_PIXEL_PERCENTAGE'].mean()
      temp = pd.DataFrame(meta_scenes_cld)
      meta_scenes_cld = temp.rename(columns={'date':'date','CLOUDY_PIXEL_PERCENTAGE':'percent_cc_scene'})
  else:
      meta_scenes_cld = meta_scenes.groupby('date')['CLOUD_COVER'].mean()
      temp = pd.DataFrame(meta_scenes_cld)
      meta_scenes_cld = temp.rename(columns={'CLOUD_COVER':'percent_cc_scene'})

  meta_df_ext = meta_df_ext.join(meta_scenes_cld)
  if dattype.startswith('S2'):
    meta_df_ext_export = meta_df_ext[['percent_coverage','percent_cc','mean_aot','percent_cc_scene']]
  else:
    meta_df_ext_export = meta_df_ext[['percent_coverage','percent_cc','percent_cc_scene']]
  return(meta_df_ext_export)


### **Part 5: Pre-fire Image Search**

Here we search for all Sentinel-2, Landsat-8 and Landsat-9 imagery.

In [24]:
# For a month's worth of data, this will take 30 seconds
cld = 100 #cloud cover threshold
pre_T1 = '2024-07-01'
pre_T2 = '2024-08-01'

## Search Sentinel-2
before_s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate(pre_T1,pre_T2).filterBounds(poly).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',cld))
s2_size = before_s2.size().getInfo()
if  s2_size !=0:
  s2_df_pre = run_search(before_s2,'S2',s2_size)
else:
  s2_df_pre = pd.DataFrame(columns=['percent_coverage','percent_cc','mean_aot','percent_cc_scene'])
  print('No Sentinel-2 Imagery Available')

## Search Landsat-8
before_l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate(pre_T1,pre_T2).filterBounds(poly).filter(ee.Filter.lt('CLOUD_COVER',cld))
l8_size = before_l8.size().getInfo()
if l8_size !=0:
  l8_df_pre = run_search(before_l8,'L8',l8_size)
else:
  l8_df_pre = pd.DataFrame(columns=['percent_coverage','percent_cc','percent_cc_scene'])
  print('No Landsat-8 Imagery Available')

## Search Landsat-9
before_l9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterDate(pre_T1,pre_T2).filterBounds(poly).filter(ee.Filter.lt('CLOUD_COVER',cld))
l9_size = before_l9.size().getInfo()
if l9_size !=0:
  l9_df_pre = run_search(before_l9,'L9',l9_size)
else:
  l9_df_pre = pd.DataFrame(columns=['percent_coverage','percent_cc','percent_cc_scene'])
  print('No Landsat-9 Imagery Available')

In [27]:
## Display data from all sensors side by side
from IPython.display import display_html
s2_df_pre_styler = s2_df_pre.style.set_table_attributes("style='display:inline'").set_caption('Sentinel-2')
l8_df_pre_styler = l8_df_pre.style.set_table_attributes("style='display:inline'").set_caption('Landsat-8')
l9_df_pre_styler = l9_df_pre.style.set_table_attributes("style='display:inline'").set_caption('Landsat-8')

display_html(s2_df_pre_styler._repr_html_()+l8_df_pre_styler._repr_html_()+l9_df_pre_styler._repr_html_(),raw=True)

,percent_coverage,percent_cc,mean_aot,percent_cc_scene
2024-07-03,99.991956,54.445642,0.129748,49.514748
2024-07-05,99.987450,0.194296,0.103372,0.246442
2024-07-08,99.991956,0.000000,0.097867,2.510186
2024-07-10,99.991956,0.000410,0.114543,0.020913
2024-07-13,99.991956,5.914910,0.093057,6.592682
2024-07-15,99.991956,2.985439,0.162335,9.067042
2024-07-18,99.991956,61.190086,0.270665,16.542571
2024-07-20,99.991956,39.499710,0.217445,4.298392
2024-07-23,99.991956,5.303989,0.099369,14.174107
2024-07-25,99.991956,98.278484,0.264089,85.244893




---



### **Search for Post-fire Images**

This script will download a BARC raster clipped to the extent of the fire perimeter polygon, as well as pre- and post-fire imagery (true color and swir RGB, 8-bit). The search criteria and scene ids will also be exported as json and text files. The root folder will be the fire number.

### **Part 11: Quicklooks and area burned by severity class**

The folder will be zipped and saved under the Files tab. Right click and download the folder to your machine.

In [ ]:
#Zip to download
zip = firenumber + '.zip'
indata = firenumber

!zip -r {zip} {indata}
files.download(zip)